##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/federated/tutorials/building_your_own_federated_learning_algorithm"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td> <a target="_blank" https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Google Colab で実行</a>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     GitHubでソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/federated/tutorials/building_your_own_federated_learning_algorithm.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 始める前に

始める前に、環境が正しくセットアップされていることを確認するために、以下を実行してください。動作しない場合は、[インストール](../install.md)ガイドで手順を確認してください。 

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

**注意**: この Colab は [最新リリースバージョン](https://github.com/tensorflow/federated#compatibility)の `tensorflow_federated` pip パッケージでの動作が確認されていますが、Tensorflow Federated プロジェクトは現在もプレリリース開発の段階にあるため、`main` では動作しない可能性があります。

# 独自の連合学習アルゴリズムを構築する

[画像分類](federated_learning_for_image_classification.ipynb)と[テキスト生成](federated_learning_for_text_generation.ipynb)のチュートリアルでは、連合学習（FL）向けにモデルとデータパイプラインをセットアップし、TFF の `tff.learning` API レイヤーを介して連合トレーニングを実行する方法を学習しました。

FL リサーチに関して言えば、これは氷山の一角に過ぎません。このチュートリアルでは、<code>tff.learning</code> API を<em>使用せずに</em>連合学習アルゴリズムを実装する方法を説明します。以下の内容を達成したいと思います。

**目標:**

- 連合学習アルゴリズムの一般的な構造を理解する。
- TFF の *Federated Core* を調べる。
- Federated Core を使用して、直接 Federated Averaging を実装する。

このチュートリアルは自己完結型ではありますが、先に、[画像分類](federated_learning_for_image_classification.ipynb)と[テキスト生成](federated_learning_for_text_generation.ipynb)のチュートリアルを読んでおくと良いでしょう。


## 入力データを準備する

まず、TFF に含まれる EMNIST データセットを読み込んで前処理します。詳細については、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルをご覧ください。

In [ ]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

データセットをモデルにフィードするには、データをフラット化して `(flattened_image_vector, label)` のタプル形式に変換します。

In [ ]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

次に、少数のクライアントを選択し、上記の前処理をデータセットに適用します。

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]

## モデルを準備する

ここでは、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルと同じモデルを使用します。このモデル（`tf.keras` で実装）には、1 つの非表示レイヤーとその後にソフトマックスレイヤーが含まれます。

In [ ]:
def create_keras_model():
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer=initializer),
      tf.keras.layers.Softmax(),
  ])

このモデルを TFF で使用するために、Keras モデルを [`tff.learning.Model`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) としてラップします。こうすることで、モデルの[フォワードパス](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#forward_pass)を TFF 内で実行し、[モデルの出力を抽出](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model#report_local_unfinalized_metrics)できるようになります。詳細については、[画像分類](federated_learning_for_image_classification.ipynb)チュートリアルもご覧ください。

In [ ]:
def model_fn():
  keras_model = create_keras_model()
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=federated_train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

`tf.keras` を使用して `tff.learning.Model` を作成しましたが、TFF でははるかに一般的なモデルがサポートされています。これらのモデルには、モデルの重みをキャプチャする以下の関連した属性があります。

- `trainable_variables`: トレーニング対象レイヤーに対応するテンソルのイテラブル。
- `non_trainable_variables`: トレーニング対象外レイヤーに対応するテンソルのイテラブル。

このチュートリアルでは、`trainable_variables` のみを使用します。（モデルにそれしか含まれていないためです！）

# 独自の連合学習アルゴリズムを構築する

`tff.learning` API では、Federated Averaging の多数のバリエーションを作成できますが、このフレームワークにうまく適合しない他の連合アルゴリズムもあります。たとえば、正則化、クリッピング、または[連合 GAN トレーニング](https://github.com/tensorflow/federated/tree/main/tensorflow_federated/python/research/gans)などのより複雑なアルゴリズムを追加することがあるかもしれません。また、[連合解析](https://ai.googleblog.com/2020/05/federated-analytics-collaborative-data.html)を実施することもあるでしょう。

こういったより高度なアルゴリズムの場合は、TFF を使用して、独自のカスタムアルゴリズムを記述しなくてはなりません。多くの場合、連合アルゴリズムには、4 つの主要コンポーネントがあります。

1. サーバーからクライアントへのブロードキャストステップ。
2. ローカルクライアントの更新ステップ。
3. クライアントからサーバーへのアップロードステップ。
4. サーバーの更新ステップ。

TFF では、連合アルゴリズムを一般的に [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess)（以降、全体を通じて `IterativeProcess` と呼びます）として表現します。これは、`initialize` と `next` 関数を含むクラスです。ここでは、`initialize` をサーバーの初期化に使用し、`next` は連合アルゴリズムの通信 1 ラウンドを実行します。FedAvg の反復プロセスがどのように見えるか、そのスケルトンを記述してみましょう。

まず、`tff.learning.Model` を作成してそのトレーニング対象重みを返すだけの初期化関数があります。

In [ ]:
def initialize_fn():
  model = model_fn()
  return model.trainable_variables

この関数は適切なようですが、後でわかるように、「TFF 計算」にするために、少しの変更を行う必要があります。

次に、`next_fn` のスケッチを記述しましょう。

In [ ]:
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = client_update(federated_dataset, server_weights_at_client)

  # The server averages these updates.
  mean_client_weights = mean(client_weights)

  # The server updates its model.
  server_weights = server_update(mean_client_weights)

  return server_weights

これらの 4 つのコンポーネントを個別に実装することに専念します。まず、純粋な TensorFlow に実装可能な部分に焦点を当てることにします。クライアントの更新ステップとサーバーの更新ステップです。


## TensorFlow のブロック 

### クライアントの更新

`tff.learning.Model` を使用して、基本的に TensorFlow モデルをトレーニングするのと同じ方法で、クライアントトレーニングを実行します。具体的に言うと、`tf.GradientTape` を使用してデータのバッチの勾配を計算してから、`client_optimizer` を使用してこれらの勾配を適用します。トレーニング対象の重みのみに注目します。


In [ ]:
@tf.function
def client_update(model, dataset, server_weights, client_optimizer):
  """Performs training (using the server model weights) on the client's dataset."""
  # Initialize the client model with the current server weights.
  client_weights = model.trainable_variables
  # Assign the server weights to the client model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        client_weights, server_weights)

  # Use the client_optimizer to update the local model.
  for batch in dataset:
    with tf.GradientTape() as tape:
      # Compute a forward pass on the batch of data
      outputs = model.forward_pass(batch)

    # Compute the corresponding gradient
    grads = tape.gradient(outputs.loss, client_weights)
    grads_and_vars = zip(grads, client_weights)

    # Apply the gradient using a client optimizer.
    client_optimizer.apply_gradients(grads_and_vars)

  return client_weights

### サーバーの更新

FedAvg のサーバー更新はクライアント更新よりも単純です。このチュートリアルでは「バニラ」Federated Averaging を実装することにしますが、ここでは、クライアントモデルの重みの平均で、サーバーモデルの重みが入れ替えられるだけです。繰り返しますが、トレーニング対象の重みのみを使用します。

In [ ]:
@tf.function
def server_update(model, mean_client_weights):
  """Updates the server model weights as the average of the client model weights."""
  model_weights = model.trainable_variables
  # Assign the mean client weights to the server model.
  tf.nest.map_structure(lambda x, y: x.assign(y),
                        model_weights, mean_client_weights)
  return model_weights

スニペットは、単に `mean_client_weights` を返すように単純化できますが、Federated Averaging の実装がより高度になれば、運動量や適合性などのより洗練されたテクニックで `mean_client_weights` を使用することができます。

**チャレンジ問題**: サーバーの重みが model_weights と mean_client_weights の中点となるように更新する `server_update` バージョンを実装してください。（注意: この種の「中点」アプローチは、[先読みオプティマイザ](https://arxiv.org/abs/1907.08610)に関する最近の作業に似ています！）

これまでは、TensorFlow コードのみで記述してきました。TFF ではすでに使い慣れた TensorFlow コードのほとんどを使用できるように設計されているためです。次では、**オーケストレーションロジック**、つまり、サーバーが何をクライアントにブロードキャストし、クライアントが何をサーバーにアップロードするのかを指示するロジックを指定しなければなりません。

この作業には、TFF の「*Federated Core*」が必要となります。

# Federated Core の導入

Federated Core（FC）は、`tff.learning` API の基盤として機能する一連の低レベルインターフェースです。ただし、これらのインターフェースは学習に制限されていません。実際、FC は分散データの分析やその他多くの計算に使用されています。

大まかに言うと、Federated Core は、TensorFlow のコードと分散通信演算子（分散和やブロードキャストなど）を組み合わせる、コンパクトに表現されたプログラムを実現する開発環境です。研究者や医師に、システムの実装情報を要求せずに（ポイントツーポイントネットワークメッセージ交換を指定するなど）、システム内の分散通信に対する明示的な制御を提供することを目標としています。

1 つの重要なポイントは、TFF がプライバシー保護のために設計されていることです。したがって、データの所在地に対する明示的な制御を行うことができるため、サーバーの中央ロケーションで望ましくないデータの蓄積が発生しないように防止できます。

## 連合データ

TFF の重要概念は「連合データ」ですが、これは、分散システムでデバイスのグループにホストされるデータアイテムのコレクションを指します（クライアントデータセット、またはサーバーモデルの重みなど）。全デバイスに渡る値のコレクション全体が単一の*連合値*として表されます。

たとえば、センサーの温度を示す浮動小数点を持つくらいアンドデバイスが複数あるとした場合、次のようにして、*連合浮動小数点*として表現することができます。

In [ ]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

連合型は、連合メンバーの型 `T`（例: `tf.float32`）とデバイスのグループ `G` で指定されます。通常 `G` は `tff.CLIENTS` または `tff.SERVER` のいずれかです。そのような連合型は、以下のように `{T}@G` として表現されます。

In [ ]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

TFF はなぜ配置にこだわるのでしょうか。TFF の主要目標は、実際の分散システムにデプロイできるコードを記述できるようにすることです。つまり、デバイスの度のサブセットがどのコードを実行し、データの異なるピースがどこに存在するかを理由づけることが重要なのです。

TFF は、*データ*、データが*配置*される場所、およびデータがどのように*変換*されるかという 3 つのことに焦点を当てています。最初の 2 つは連合型に含まれますが、最後の項目は*連合計算*に含まれています。

## 連合計算

TFF は強力に型付けされた関数型プログラミング環境で、その基本単位は*連合計算*です。これらは、連合値を入力として受け入れ、連合値を出力として返すロジックです。

たとえば、クライアントセンサーの温度を平均化するとした場合、以下のように（連合浮動小数点を使用して）定義することができます。

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(client_temperatures):
  return tff.federated_mean(client_temperatures)

これが TensorFlow の `tf.function` デコレータとどのように異なるのか疑問に思うかもしれません。ここで重要なのは、`tff.federated_computation` が生成するコードは、TensorFlow コードでも Python コードでもないということです。つまり、これは内部プラットフォーム非依存型の*グルー言語*による分散システムの仕様です。

複雑に聞こえるかもしれませんが、TFF 計算を、十分に定義づけされた型シグネチャ付きの関数と捉えることができます。これらの型シグネチャは直接クエリすることができます。

In [ ]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

この `tff.federated_computation` は、連合型 `<float>@CLIENTS` の引数を受け入れ、連合型 `<float>@SERVER` の出力を返します。連合計算もサーバーからクライアント、クライアントからクライアント、またはサーバーからサーバーに移動することができます。また、型シグネチャが一致する限り、通常の関数のように作成することができます。

開発を支援するために、TFF では `tff.federated_computation` を Python 関数として呼び出すことができます。たとえば、以下を呼び出すことが可能です。

In [ ]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

## 非 eager 計算と TensorFlow

注意しておかなければならない重要な制限事項が 2 つあります。1 つは、Python インタープリタが `tff.federated_computation` デコレータに遭遇すると、関数のトレースが一度行われ、以降で使用できるようにシリアル化されるという制限です。連合学習の分散化の性質により、これはリモート実行環境などの別の場所で今後起きるかもしれません。そのため、TFF 計算は基本的に*非 eager* で行われます。この動作は、TensorFlow の [`tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) デコレータの動作にやや似ています。

2 つ目は、連合計算には連合演算子（`tff.federated_mean` など）しか使用できず、TensorFlow 演算子を含めることはできないという制限です。TensorFlow コードは `tff.tf_computation` でデコレートされたブロックに閉じ込められている必要があります。ほとんどの一般的な TensorFlow コードは、数字を取得してそれに `0.5` を追加する以下の関数のように、直接デコレートすることができます。

In [ ]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

これらにも型シグネチャがありますが、*位置付けされていません*。たとえば、以下を呼び出すことができます。

In [ ]:
str(add_half.type_signature)

'(float32 -> float32)'

ここでは、`tff.federated_computation` と `tff.tf_computation` の重要な違いがわかります。前者は明示的な位置づけがあり、後者にはありません。

連合計算では配置を指定することで、`tff.tf_computation` ブロックを使用できます。クライアントの連合浮動小数点のみに半分を追加する関数を作成してみましょう。これは、配置を保持しながら特定の `tff.tf_computation` を適用する `tff.federated_map` を使って行います。

In [ ]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

この関数はほぼ `add_half` と同じですが、`tff.CLIENTS` に配置されている値のみを受け入れ、同じ配置の値を返します。これは型シグネチャで確認できます。

In [ ]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

要約:

- TFF は連合値で演算します。
- 各連合値には、*型*（例: <code>tf.float32</code>）と<em>配置</em>（例:  <code>tff.CLIENTS</code>）を持つ<em>連合型</em>があります。
- 連合値は、*連合計算*を使って変換できますが、`tff.federated_computation` と連合型シグネチャでデコレートされている必要があります。
- TensorFlow コードは `tff.tf_computation` デコレータを持つブロックに格納されている必要があります。
- その上で、これらのブロックを連合計算に組み込むことができます。


# 独自の連合学習アルゴリズムを構築する（再確認）

Federated Core を垣間見たところで、独自の連合学習アルゴリズムを作成することにしましょう。上記では、アルゴリズムに `initialize_fn` と `next_fn` を定義したことを思い出してください。`next_fn` は純粋な TensorFlow コードを使用して定義した `client_update` と `server_update` を利用します。

ただし、アルゴリズムを連合計算にするには、`next_fn` と `initialize_fn` がそれぞれ `tff.federated_computations` である必要があります。

## TensorFlow Federated ブロック 

### 初期化計算を作成する

初期化関数は非常に単純です。`model_fn` を使用してモデルを作成します。ただし、`tff.tf_computation` を使用して、TensorFlow コードを分けておく必要があったことを思い出しましょう。

In [ ]:
@tff.tf_computation
def server_init():
  model = model_fn()
  return model.trainable_variables

次に、`tff.federated_value` を使用して、これを直接連合計算に渡します。

In [ ]:
@tff.federated_computation
def initialize_fn():
  return tff.federated_value(server_init(), tff.SERVER)

### `next_fn` を作成する

クライアントサーバーの更新コードを使って、実際のアルゴリズムを作成できるようになりました。まず、`client_update` を、クライアントデータセットとサーバーの重みを受け入れて、更新されたクライアントの重みテンソルを出力する `tff.tf_computation` に変換します。

関数を適切にデコレートするために、対応する型が必要です。幸いにも、サーバーの重みの型は、モデルから直接抽出することができます。

In [ ]:
whimsy_model = model_fn()
tf_dataset_type = tff.SequenceType(whimsy_model.input_spec)

データセットの型シグネチャを確認しましょう。28 x 28 の画像（整数のラベル付き）を取得して、平坦化したことを思い出してください。

In [ ]:
str(tf_dataset_type)

'<float32[?,784],int32[?,1]>*'

また、上記の `server_init` 関数を使用して、モデルの重みの型を抽出することもできます。

In [ ]:
model_weights_type = server_init.type_signature.result

型シグネチャを調べると、モデルのアーキテクチャを確認できます！

In [ ]:
str(model_weights_type)

'<float32[784,10],float32[10]>'

これで、クライアントの更新用の `tff.tf_computation` を作成できるようになりました。

In [ ]:
@tff.tf_computation(tf_dataset_type, model_weights_type)
def client_update_fn(tf_dataset, server_weights):
  model = model_fn()
  client_optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  return client_update(model, tf_dataset, server_weights, client_optimizer)

サーバー更新バージョンの `tff.tf_computation` は、すでに抽出した型を使用して、同じようにして定義することができます。

In [ ]:
@tff.tf_computation(model_weights_type)
def server_update_fn(mean_client_weights):
  model = model_fn()
  return server_update(model, mean_client_weights)

最後に、このすべてをまとめる `tff.federated_computation` を作成する必要があります。この関数は、サーバーの重みに対応する値（配置が <code>tff.SERVER</code> のもの）とクライアントデータセットに対応する値（配置が `tff.CLIENTS` のもの）の 2 つの<em>連合値</em>を受け入れます。

これら両方の型が上記で定義されているところに注意してください！`tff.FederatedType` を使用して適切な配置を指定することだけが必要です。

In [ ]:
federated_server_type = tff.FederatedType(model_weights_type, tff.SERVER)
federated_dataset_type = tff.FederatedType(tf_dataset_type, tff.CLIENTS)

FL アルゴリズムの 4 つの要素を覚えていますか？

1. サーバーからクライアントへのブロードキャストステップ。
2. ローカルクライアントの更新ステップ。
3. クライアントからサーバーへのアップロードステップ。
4. サーバーの更新ステップ。

上記の構築が完了したので、各パーツを TFF コードの単一の行としてコンパクトに表現することができます。連合型などを指定して手間をかけたのは、この単純さを実現するためです！

In [ ]:
@tff.federated_computation(federated_server_type, federated_dataset_type)
def next_fn(server_weights, federated_dataset):
  # Broadcast the server weights to the clients.
  server_weights_at_client = tff.federated_broadcast(server_weights)

  # Each client computes their updated weights.
  client_weights = tff.federated_map(
      client_update_fn, (federated_dataset, server_weights_at_client))
  
  # The server averages these updates.
  mean_client_weights = tff.federated_mean(client_weights)

  # The server updates its model.
  server_weights = tff.federated_map(server_update_fn, mean_client_weights)

  return server_weights

両方のアルゴリズム初期化と、アルゴリズムの 1 つのステップの実行を行うめの `tff.federated_computation` を用意できました。このアルゴリズムを終了するために、これらを `tff.templates.IterativeProcess` に渡します。

In [ ]:
federated_algorithm = tff.templates.IterativeProcess(
    initialize_fn=initialize_fn,
    next_fn=next_fn
)

反復プロセスの <code>initialize</code> と `next` 関数の<em>型シグネチャ</em>を見てみましょう。

In [ ]:
str(federated_algorithm.initialize.type_signature)

'( -> <float32[784,10],float32[10]>@SERVER)'

これは、`federated_algorithm.initialize` が単一レイヤーモデル（784 x10 の重み行列と 10 バイアスユニット）を返す引数なし関数であることを反映しています。

In [ ]:
str(federated_algorithm.next.type_signature)

'(<server_weights=<float32[784,10],float32[10]>@SERVER,federated_dataset={<float32[?,784],int32[?,1]>*}@CLIENTS> -> <float32[784,10],float32[10]>@SERVER)'

ここでは、`federated_algorithm.next` がサーバーモデルとクライアントデータを受け入れて、更新されたサーバーモデルを返すことがわかります。

## アルゴリズムを評価する

数ラウンドほど実行し、損失がどのように変化するかを見てみましょう。まず、2 つ目のチュートリアルで説明した *centralized* アプローチを使って、評価関数を定義します。

まず、中央の評価データセットを作成してから、トレーニングデータに使用したのと同じ前処理を適用します。

In [ ]:
central_emnist_test = emnist_test.create_tf_dataset_from_all_clients()
central_emnist_test = preprocess(central_emnist_test)

次に、サーバーの状態を受け入れる関数を記述し、Keras を使用してテストデータセットで評価します。`tf.Keras` の使用に慣れているのであれば、これも見慣れているかもしれませんが、`set_weights` の使用に注意してください！

In [ ]:
def evaluate(server_state):
  keras_model = create_keras_model()
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  
  )
  keras_model.set_weights(server_state)
  keras_model.evaluate(central_emnist_test)

では、アルゴリズムを初期化して、テストセットを評価してみましょう。

In [ ]:
server_state = federated_algorithm.initialize()
evaluate(server_state)

2042/2042 [==============================] - 2s 767us/step - loss: 2.8479 - sparse_categorical_accuracy: 0.1027


数ラウンド程度トレーニングし、何かが変化するかどうかを確認しましょう。

In [ ]:
for round in range(15):
  server_state = federated_algorithm.next(server_state, federated_train_data)

In [ ]:
evaluate(server_state)

2042/2042 [==============================] - 2s 738us/step - loss: 2.5867 - sparse_categorical_accuracy: 0.0980


損失関数がわずかに減少しているのがわかります。小さなジャンプではありますが、トレーニングは 15 ラウンドしか実行しておらず、クライアントのサブセットも小さくなっています。結果をよく理解するには、数千ラウンドでないにしても、数百ラウンドは実行する必要があるかもしれません。

## アルゴリズムを変更する

ここまでたどり着いたところで、手を休め、これまで達成したことを考えてみましょう。純粋な TensorFlow コード（クライアントとサーバーの更新用）と TFF の Federated Core の連合計算を組み合わせることで、Federated Averaging を直接実装しました。

単に上記の内容を変更するだけで、さらに洗練された学習を実行することができます。具体的には、上記の純粋な TF コードを編集することで、クライアントがトレーニングを実行する方法またはサーバーがモデルを更新する方法を変更することができます。

**課題:** <code>client_update</code> 関数に<a>勾配クリップ</a>を追加してください。


変化をより大きくする場合は、さらに多くのデータをサーバーに保存してブロードキャストすることも可能です。たとえば、サーバーはクライアントの学習率も保存して、経時的に減衰させることもできます！これには、上記の `tff.tf_computation` 呼び出しに使用されている型シグネチャに変更を加える必要があります。

**さらに難易度の高いチャレンジ問題:** クライアントの学習率の減衰を使用して Federated Averaging を実装してください。

この時点で、このフレームワークに実装できるものにどれくらいの柔軟性があるかがわかり始めたところでしょう。考え方（上記のさらに難易度の高いチャレンジ問題の解答を含む）については、[`tff.learning.algorithms.build_weighted_fed_avg`](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) のソースコードを確認するか、TFF を使用したさまざまな[リサーチプロジェクト](https://github.com/google-research/federated)をご覧ください。